# The mirroring is quite computational expensive therefore the diagonal blocks get precomputed

In [1]:
import cloudpickle
from scipy.sparse import dok_matrix
from scipy.sparse import tril
import scipy
import re
import time

def load_dict(path,zeilen,spalten):
    if(zeilen >= spalten):
        template = "blockcounts_{i}_{j}".format(i=zeilen,j=spalten)
    else:
        template = "blockcounts_{i}_{j}".format(i=spalten,j=zeilen)

    file_path = path + '\\' + template
    co_occurences = {}
    
    
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            match = re.match('\(([0-9]{1,}), ([0-9]{1,})\):([0-9.]{1,})',line)
            x = int(match.group(1))
            y = int(match.group(2))
            count = float(match.group(3))
            if (x,y) in co_occurences:
                co_occurences[(x,y)] += count
            else:
                co_occurences[(x,y)] = count
    
    return co_occurences


def load_co_occurence(path,zeilen,spalten):
    co_occurences = load_dict(path,zeilen,spalten)
    coocurrence = dok_matrix((20000,20000),dtype='d')
    
    coocurrence._update(co_occurences) # dok_matrix updates #7673 pull request

    if spalten > zeilen :
        raise Exception("You sha'll not get symmetrical context")
        #may need to fix this in the future due to other non symmetrical feature
    
    if spalten == zeilen:
        print('mirroring')
        #print(coocurrence.toarray())
        coocurrence = coocurrence + tril(coocurrence,k=-1).transpose()
    return coocurrence

In [2]:
import re
import os
import h5py
import numpy as np
import tensorflow as tf

# The default block length of 20000 must be divisible by this number. 
split_length = 4 #2,4,5,10
size = int(20000/split_length)

path = 'E:\\tmp\\coocurrence_blocks'
path_hdf = r'E:\\hdf_base_coocurrence_2021_'+str(size)+'\\'
regex = r'blockcounts_([0-9]{1,})_([0-9]{1,})'

for file_name in os.listdir(path):
    match = re.match(regex, file_name)
    i,j   = match.group(1), match.group(2)
    i,j   = int(i)        , int(j)
    print('output for progress ' + str(i)+','+str(j))
    if i< j:
        continue
    

    co_occurence = load_co_occurence(path,i,j)
    co_occurence  = co_occurence.toarray()
    if(i == j):
        np.fill_diagonal(co_occurence,0)
    
    for sub_i in range(split_length):
        for sub_j in range(split_length):
            a,b = i*split_length+sub_i , j*split_length+sub_j
            print(a,b)
            filename = 'tf_cooccurence_{a}_{b}.tensor'.format(a = a,b = b)
            tf_subcur = tf.convert_to_tensor(co_occurence[sub_i*size:(sub_i+1)*size,sub_j*size:(sub_j+1)*size],dtype=tf.float32)
            with open(path_hdf + filename, 'wb+') as file:
                cloudpickle.dump(tf_subcur, file)
        
    #print(co_occurence_hdf)

output for progress 0,0
mirroring
0 0
0 1
0 2
0 3
1 0
1 1
1 2
1 3
2 0
2 1
2 2
2 3
3 0
3 1
3 2
3 3
output for progress 10,0
40 0
40 1
40 2
40 3
41 0
41 1
41 2
41 3
42 0
42 1
42 2
42 3
43 0
43 1
43 2
43 3
output for progress 10,1
40 4
40 5
40 6
40 7
41 4
41 5
41 6
41 7
42 4
42 5
42 6
42 7
43 4
43 5
43 6
43 7
output for progress 10,10
mirroring
40 40
40 41
40 42
40 43
41 40
41 41
41 42
41 43
42 40
42 41
42 42
42 43
43 40
43 41
43 42
43 43
output for progress 10,2
40 8
40 9
40 10
40 11
41 8
41 9
41 10
41 11
42 8
42 9
42 10
42 11
43 8
43 9
43 10
43 11
output for progress 10,3
40 12
40 13
40 14
40 15
41 12
41 13
41 14
41 15
42 12
42 13
42 14
42 15
43 12
43 13
43 14
43 15
output for progress 10,4
40 16
40 17
40 18
40 19
41 16
41 17
41 18
41 19
42 16
42 17
42 18
42 19
43 16
43 17
43 18
43 19
output for progress 10,5
40 20
40 21
40 22
40 23
41 20
41 21
41 22
41 23
42 20
42 21
42 22
42 23
43 20
43 21
43 22
43 23
output for progress 10,6
40 24
40 25
40 26
40 27
41 24
41 25
41 26
41 27
42 24
42 25

22 21
22 22
22 23
23 20
23 21
23 22
23 23
output for progress 6,0
24 0
24 1
24 2
24 3
25 0
25 1
25 2
25 3
26 0
26 1
26 2
26 3
27 0
27 1
27 2
27 3
output for progress 6,1
24 4
24 5
24 6
24 7
25 4
25 5
25 6
25 7
26 4
26 5
26 6
26 7
27 4
27 5
27 6
27 7
output for progress 6,2
24 8
24 9
24 10
24 11
25 8
25 9
25 10
25 11
26 8
26 9
26 10
26 11
27 8
27 9
27 10
27 11
output for progress 6,3
24 12
24 13
24 14
24 15
25 12
25 13
25 14
25 15
26 12
26 13
26 14
26 15
27 12
27 13
27 14
27 15
output for progress 6,4
24 16
24 17
24 18
24 19
25 16
25 17
25 18
25 19
26 16
26 17
26 18
26 19
27 16
27 17
27 18
27 19
output for progress 6,5
24 20
24 21
24 22
24 23
25 20
25 21
25 22
25 23
26 20
26 21
26 22
26 23
27 20
27 21
27 22
27 23
output for progress 6,6
mirroring
24 24
24 25
24 26
24 27
25 24
25 25
25 26
25 27
26 24
26 25
26 26
26 27
27 24
27 25
27 26
27 27
output for progress 7,0
28 0
28 1
28 2
28 3
29 0
29 1
29 2
29 3
30 0
30 1
30 2
30 3
31 0
31 1
31 2
31 3
output for progress 7,1
28 4
28 5
28 6
28 7
